# Run FIRST

In [20]:
### VARIABLES ###

#python variables
REF_GENS = {
    'Chicken': '/beegfs/work/graaf20/Databases/Ref_gens/Chicken/GCA_024206055.2_GGswu_genomic.fna',
    'Corn': '/beegfs/work/graaf20/Databases/Ref_gens/Corn/GCA_902167145.1_Zm-B73-REFERENCE-NAM-5.0_genomic.fna'
}


#shared between bash and python
VARS = (
    'WS=/beegfs/work/graaf20/ChickenPhages\n'
    'DB=/beegfs/work/graaf20/Databases/\n'
    'DATA=Data\n'
    'RES=Results\n'
    'ASS=Results/Assemlies_MegaHit\n'
    'MH=MegaHit_temp\n'
    'FIG=Figures\n'
    'LOGS=Logs\n'
    'RR=Raw_reads\n'
    'CR=Clean_reads\n'
)

#METADATA = pd.read_csv('metadata.tsv', sep='\t', index_col=0)

CONDA = '$HOME/miniconda3/etc/profile.d/conda.sh'


##################### DO NOT CHANGE ################################
import pandas as pd
import numpy as np
import os

#extract bash variables for python
for var in VARS.strip().split('\n'):
    vars()[var.split('=')[0]]=var.split('=')[1]

!mkdir -p $WS/$CR $WS/$LOGS $WS/$TABS $WS/$RES $WS/$FIG
    
#Binac function
params = '''
#PBS -l nodes={nodes}:ppn={ppn}
#PBS -l walltime={time}
#PBS -l mem={mem}gb
#PBS -N {name}.sh
#PBS -o {logdir}
#PBS -e {logdir}
#PBS -q {queue}
#PBS -S /bin/bash
export TMPDIR=$TMPDIR
''' 

source = f'source {CONDA}'

COMM = '#Custom variables and commands\n{todo}'
STRING = '\n'.join([params, source, VARS, COMM])

# binac function that will submit the job to server
def binac(nodes=1, ppn=1, time='00:15:00', mem=2, name='myjob', logdir=WS,
          queue='tiny', todo='echo "Command"', STRING=STRING, ):
    text = STRING
    if '{nodes}' in STRING:
        text = STRING.format(nodes=nodes, ppn=ppn, time=time, logdir=f'{WS}/{logdir}',
               mem=mem, name=name, queue=queue, todo=todo)
    script = f'{WS}/{logdir}/{name}.sh'
    !mkdir -p $WS/$logdir
    with open(script, "w") as text_file:
        text_file.write(text)
    !chmod +x $script
    !qsub $script
    
###############################################################################################

bad = ['SRR5280393', 'ERR2241736', 'ERR2241917', 'SRR12730717', 'SRR7683044',
       'SRR7457769', 'SRR9113694', 'ERR2241801', 'SRR17836573', 'SRR7685268', 
       'SRR9113697', 'SRR9113700', 'SRR5280759', 'SRR17836591', 'SRR7457773', 
       'SRR7683048', 'ERR2241740', 'ERR2241741', 'SRR17836604', 'SRR7683050', 
       'SRR7457774', 'ERR2241809', 'SRR5282097', 'SRR12730726', 'SRR7683052',
       'ERR2241763', 'ERR2241943', 'SRR7685282', 'SRR17836559', 'SRR7457783',
       'ERR2241850', 'ERR2241735', 'ERR2241764', 'SRR12730743', 'SRR7457734', 
       'ERR2241631', 'SRR7683056', 'SRR7457738', 'SRR7683059',  'ERR2241635',
       'SRR7457741', 'ERR2241791', 'ERR2241795', 'SRR7457745',  'ERR2241689', 
       'SRR7685255', 'SRR7685256', 'ERR2241698', 'SRR7457755', ] 

# get batches and locals
df = pd.read_csv(f'{WS}/Data/MAGs_dataset_complete.csv', index_col=0, na_filter = False)
df.BioProject = df.BioProject.str.replace('NA', 'Local')
#locs = df.loc[df.BioProject == 'Local', 'Run'].unique().tolist()
#sras = df.loc[df.BioProject != 'Local', 'Run'].unique().tolist()
sras = df.loc[~df.Run.isin(bad), 'Run'].unique().tolist()
b = 30
batches = {f'batch{i+1}-{i+b}': sras[i:i + b] for i in range(0, len(sras), b)}

# Create list of file indexes to access it later
cleaning = !rm -r $WS/*/*/*/.ipynb_ch* $WS/*/*/.ipynb_ch* $WS/*/.ipynb_ch* $WS/.ipynb_ch*

raws = []
if os.path.exists(f'{WS}/{RR}'): 
    raws = [f.rsplit('_', 1)[0] for f in os.listdir(f'{WS}/{RR}') if '.gz' in f]
    raws = sorted(list(set(raws)))
    print('Number of raw samples: ', len(raws))

cleans = []
if os.path.exists(f'{WS}/{CR}'): 
    cleans = [f.rsplit('_', 1)[0] for f in os.listdir(f'{WS}/{CR}') if '.gz' in f]
    cleans = sorted(list(set(cleans)))
    print('Number of clean samples: ', len(cleans))
    
asses = []
if os.path.exists(f'{WS}/{ASS}'): 
    asses = [f.rsplit('.', 1)[0] for f in os.listdir(f'{WS}/{ASS}') if '.fa' in f]
    asses = sorted(list(set(asses)))
    print('Number of assembles: ', len(asses))
    
permissions = !chmod +rwx $BIN/*



Number of raw samples:  0
Number of clean samples:  1080
Number of assembles:  1446


In [238]:
len(sras)

1515

In [239]:
batches

{'batch1-30': ['SRR12730711',
  'SRR12730712',
  'SRR12730713',
  'SRR12730714',
  'SRR12730715',
  'SRR12730716',
  'SRR12730718',
  'SRR12730719',
  'SRR12730720',
  'SRR12730721',
  'SRR12730722',
  'SRR12730723',
  'SRR12730724',
  'SRR12730725',
  'SRR12730727',
  'SRR12730728',
  'SRR12730729',
  'SRR12730730',
  'SRR12730731',
  'SRR12730732',
  'SRR12730733',
  'SRR12730734',
  'SRR12730735',
  'SRR12730736',
  'SRR12730737',
  'SRR12730738',
  'SRR12730739',
  'SRR12730740',
  'SRR12730741',
  'SRR12730742'],
 'batch31-60': ['SRR12730744',
  'SRR12730745',
  'SRR12730746',
  'SRR12730747',
  'SRR12730748',
  'SRR12730749',
  'SRR12730750',
  'SRR8840688',
  'SRR8840689',
  'SRR8840690',
  'SRR8840691',
  'SRR8840692',
  'SRR8840693',
  'SRR8840694',
  'SRR6323082',
  'SRR6323083',
  'SRR6323084',
  'SRR6323085',
  'SRR6323086',
  'SRR6323087',
  'SRR6323088',
  'SRR6323089',
  'SRR6323090',
  'SRR6323091',
  'SRR6323092',
  'SRR6323093',
  'SRR6323094',
  'SRR6323095',
  'SRR6

# SRA-Toolkit

In [95]:
#use only 1 thread or bad things may happen (Binac will freeze)!

THREADS = 2
logdir = f'{LOGS}/SRAs'

todo = '''
conda activate QC

cd $WS
mkdir -p sra_temp

echo "Processing {BATCH}"
sras=({IDS})
for sra in ${{sras[@]}}; do

    #skip processed files
    if [ -f $RR/${{sra}}_1.fastq.gz ]; then
        echo "$sra already exists. Skipping."
        continue
    fi
    
    if [ -f $RR/${{sra}}.fastq.gz ]; then
        echo "$sra already exists. Skipping."
        continue
    fi
    
    if [ -f $CR/${{sra}}_1.fq.gz ]; then
        echo "$sra already exists. Skipping."
        continue
    fi
    
    if [ -f $CR/${{sra}}.fq.gz ]; then
        echo "$sra already exists. Skipping."
        continue
    fi
    
    #download files
    echo "Prefetch $sra"
    prefetch $sra -O sra_temp --verify yes --max-size 50G

    echo "Convert to fastq $sra"
    fasterq-dump sra_temp/$sra --split-files --outdir $RR --threads 1
    rm -r sra_temp/$sra

    echo "Gzip $sra"
    pigz $RR/${{sra}}_*.fastq
    pigz $RR/${{sra}}.fastq

    
done
echo "All done, check the outputs"
'''

!mkdir -p $WS/$RR

for batch, ids in batches.items():
    if batch not in ['batch1001-1020']:
        continue
        
    IDS = ' '.join([f"'{l}'" for l in [i for i in ids if i not in bad]])
    binac(ppn=THREADS, time='07:00:00:00', mem=16, name=f'{batch}_SRA', queue='long', 
          logdir=logdir, todo=todo.format(BATCH=batch, IDS=IDS))


11366442


In [33]:
# Summarize raw data

df['Status'] = 'Downloaded'
df.loc[df.Run.isin(bad), 'Status'] = 'Broken'

df.to_csv(f'{WS}/Data/MAGs_dataset_complete.csv')

In [37]:
len(df.loc[df.Status == 'Downloaded'])

1515

# Local samples

In [ ]:
#copy assembled contigs for UHO samples and raw data for LH samples

!mkdir -p $WS/$ASS

for loc in locs:
    if loc.startswith('UHO'):
        continue
        src = f'~/Beeg/PFOWL2-3/Results/MegaHit/{loc}/final.contigs.fa'
        new = f'{WS}/{ASS}/{loc}.fa'
        
        !cp $src $new
        
    if loc.startswith('LH'):
        for i in range(1,3):
            src = f'/beegfs/work/croth/LH1_metagenomes/Clean_reads/{loc}_{i}.fq.gz'
            new = f'{WS}/{RR}/{loc}_{i}.fastq.gz'
            if os.path.exists(new) or os.path.exists(f'{WS}/{CR}/{loc}_{i}.fq.gz'):
                continue
            
            !cp $src $new

In [163]:
locs

['UHO1',
 'UHO2',
 'UHO3',
 'UHO4',
 'UHO5',
 'UHO6',
 'UHO7',
 'UHO8',
 'UHO9',
 'UHO10',
 'UHO11',
 'UHO12',
 'UHO13',
 'UHO14',
 'UHO15',
 'UHO16',
 'UHO17',
 'UHO18',
 'UHO19',
 'UHO20',
 'UHO21',
 'UHO22',
 'UHO23',
 'UHO24',
 'UHO25',
 'UHO26',
 'UHO27',
 'UHO28',
 'UHO29',
 'UHO30',
 'UHO31',
 'UHO32',
 'UHO33',
 'UHO34',
 'UHO35',
 'UHO36',
 'UHO37',
 'UHO38',
 'UHO39',
 'UHO40',
 'UHO41',
 'UHO42',
 'UHO43',
 'UHO44',
 'UHO45',
 'UHO46',
 'UHO47',
 'UHO48',
 'UHO49',
 'UHO50',
 'UHO51',
 'UHO52',
 'UHO53',
 'UHO54',
 'UHO55',
 'UHO56',
 'UHO57',
 'UHO58',
 'UHO59',
 'UHO60',
 'UHO61',
 'UHO62',
 'UHO63',
 'UHO64',
 'UHO65',
 'UHO66',
 'UHO67',
 'UHO68',
 'UHO69',
 'UHO70',
 'UHO71',
 'UHO72',
 'UHO73',
 'UHO74',
 'UHO75',
 'UHO76',
 'UHO77',
 'UHO78',
 'UHO79',
 'UHO80',
 'UHO81',
 'UHO82',
 'UHO83',
 'UHO84',
 'UHO85',
 'UHO86',
 'UHO87',
 'UHO88',
 'UHO89',
 'UHO90',
 'UHO91',
 'UHO92',
 'UHO93',
 'UHO94',
 'UHO95',
 'UHO96',
 'UHO97',
 'UHO98',
 'UHO99',
 'UHO100',
 'UHO101

# ReadQC and host removal

In [51]:
#create index

THREADS = 8

todo = '''
conda activate QC

cd {ws}
bowtie2-build {refer} {index} --threads {THREADS}
'''

for ind, ref in REF_GENS.items():
    ws,fa = ref.rsplit('/', 1)
    binac(ppn=THREADS, time='08:00:00', mem=90, name=f'{ind}_Ref', queue='short', 
          logdir=LOGS, todo=todo.format(ws=ws, refer=fa, index=ind, THREADS=THREADS))

11130446
11130447


In [107]:
#Read QC and host DNA removal

THREADS = 8
logdir = f"{LOGS}/QC"

todo = '''
conda activate QC

echo "Processing {BATCH}"
sras=({IDS})
for name in ${{sras[@]}}; do
    cd $WS
    ref_host={ref_host}
    ind_host={ind_host}
    ref_feed={ref_feed}
    ind_feed={ind_feed}
    preQC=$DATA/FastQC/preQC/$name
    postQC=$DATA/FastQC/postQC/$name
    outdir=$CR/$name
    
    echo
    echo "############################################################"
    echo "Processing $name "
    echo "Check if reads are paired or single" 
    
    if [ -f $RR/${{name}}_1.f*q.gz ]; then
        echo "$name are paired reads."
        RR1=$RR/${{name}}_1.f*q.gz
        RR2=$RR/${{name}}_2.f*q.gz
        CR1=$CR/${{name}}_1.fq.gz
        CR2=$CR/${{name}}_2.fq.gz
    
        #skip processed files
        if [ -f $CR1 ]; then
            echo "$name already cleaned. Skipping."
            continue
        fi
        
        rm -rf $outdir $preQC $postQC $CR1 $CR2
        mkdir -p $preQC $postQC $outdir 
    
        # Fastq report before QC
        echo "Report before QC..."
        fastqc -q -t {THREADS} -o $preQC -f fastq $RR1 $RR2
        
        echo
        echo "Running Trim-Galore for QC"
        echo   
        cd $outdir
        trim_galore --paired $WS/$RR1 $WS/$RR2 -j {THREADS}
    
        # Fastq report after QC
        echo "Report after QC..."
        cd
        cd $WS
        fastqc -q -t {THREADS} -o $postQC -f fastq $outdir/*_val_1.f*q.gz $outdir/*_val_2.f*q.gz
        
        echo
        echo "Removing Host and Feed DNA from reads"
        echo "Mapping reads to the host reference..."
        cd
        cd $ref_host
        bowtie2 -p {THREADS} -x $ind_host -1 $WS/$outdir/*_val_1.f*q.gz -2 $WS/$outdir/*_val_2.f*q.gz \
        --un-conc-gz $WS/$outdir/no_host > $WS/$outdir/host.sam
        echo "Host DNA removed"
        
        echo "Mapping reads to the feed reference..."
        cd
        cd $ref_feed
        bowtie2 -p {THREADS} -x $ind_feed -1 $WS/$outdir/no_host.1 -2 $WS/$outdir/no_host.2 \
        --un-conc-gz $WS/$outdir/no_feed > $WS/$outdir/feed.sam
        echo "Feed DNA removed"
        
        echo
        echo "Move and clean!"
        cd
        cd $WS
        mv $outdir/no_feed.1 $CR1
        mv $outdir/no_feed.2 $CR2
        rm -rf $outdir
    
        #remove processed files
        if [ -f $CR1 ]; then
            echo "Removing raw read since $name successfully cleaned."
            rm $RR1 $RR2
        fi
    fi

    if [ -f $RR/${{name}}.f*q.gz ]; then
        echo "$name are single reads."
        RR1=$RR/${{name}}.f*q.gz
        CR1=$CR/${{name}}.fq.gz
    
        #skip processed files
        if [ -f $CR1 ]; then
            echo "$name already cleaned. Skipping."
            continue
        fi
        
        rm -rf $outdir $preQC $postQC $CR1 $CR2
        mkdir -p $preQC $postQC $outdir 
    
        # Fastq report before QC
        echo "Report before QC..."
        fastqc -q -t {THREADS} -o $preQC -f fastq $RR1
        
        echo
        echo "Running Trim-Galore for QC"
        echo   
        cd $outdir
        trim_galore $WS/$RR1 -j {THREADS}
    
        # Fastq report after QC
        echo "Report after QC..."
        cd
        cd $WS
        fastqc -q -t {THREADS} -o $postQC -f fastq $outdir/*_trimmed.f*q.gz
        
        echo
        echo "Removing Host and Feed DNA from reads"
        echo "Mapping reads to the host reference..."
        cd
        cd $ref_host
        bowtie2 -p {THREADS} -x $ind_host -U $WS/$outdir/*_trimmed.f*q.gz \
        --un-gz $WS/$outdir/no_host > $WS/$outdir/host.sam
        echo "Host DNA removed"
        
        echo "Mapping reads to the feed reference..."
        cd
        cd $ref_feed
        bowtie2 -p {THREADS} -x $ind_feed -U $WS/$outdir/no_host \
        --un-gz $WS/$outdir/no_feed > $WS/$outdir/feed.sam
        echo "Feed DNA removed"
        
        echo
        echo "Move and clean!"
        cd
        cd $WS
        mv $outdir/no_feed $CR1
        rm -rf $outdir
    
        #remove processed files
        if [ -f $CR1 ]; then
            echo "Removing raw read since $name successfully cleaned."
            rm $RR1
        fi
        
    else
        Echo "There is something wrong with that sample (neither paired reads or single read were found)"
        continue
        
    fi
done

echo
echo "############################################################"
echo "###                   All done! Enjoy!                   ###"
echo "### Citations:                                           ###"
echo "### 1. Cutadapt: https://doi.org/10.14806/ej.17.1.200    ###"
echo "### 2. Bowtie: https://doi.org/10.1038/nmeth.1923        ###"
echo "### 3. FastQC (optional):                                ###"
echo "###    https://doi.org/10.1038/nmeth.1923                ###"
echo "### 4. TrimGalore (optional):                            ###"
echo "###    https://github.com/FelixKrueger/TrimGalore        ###"
echo "############################################################"
'''

#get indexes for Host/Feed genomes
inds, refs = [], []
for ind, ref in REF_GENS.items():
    inds.append(ind)
    refs.append(ref.rsplit('/', 1)[0])


#launch samples
for batch, ids in batches.items():
    if batch not in ['batch1001-1020']:
        continue
        
    IDS = ' '.join([f"'{l}'" for l in [i for i in ids if i not in bad]])
            
    binac(ppn=THREADS, time='07:00:00:00', mem=120, name=f'{batch}_QC', queue='long', 
              logdir=logdir, todo=todo.format(BATCH=batch, IDS=IDS, ref_host=refs[0], 
              ind_host=inds[0], ref_feed=refs[1], ind_feed=inds[1], THREADS=THREADS))


11366477


In [ ]:
#keep output reports 

logs = f"{WS}/{LOGS}/QC"

!mkdir {WS}/$logs
!mv *_QC.sh.* {WS}/$logs/

In [48]:
logs = f"{WS}/{LOGS}/QC"
summ = pd.DataFrame()
summ.index.name='SampleID'
for clean in cleans:
    if clean in [""]:
        continue
    file = [f for f in os.listdir(logs) if f.startswith(f"{clean}_QC.sh.e")][0]
    with open(f'{logs}/{file}') as f:
        txt = f.read().split('Total number of sequences analysed: ')[-1]

        #Get QC stats
        total = float(txt.split('\n')[0])
        bad = float(txt.split('bp): ')[-1].split(' (')[0])
        
        #Get Host/Feed DNA
        no_host = float(txt.split('-\n    ')[1].split(' ')[0])
        no_feed = float(txt.split('-\n    ')[3].split(' ')[0])
        
    summ.loc[clean, ['Total', 'PassedQC', 'NoHost', 'NoFeed']] = [total, total-bad, no_host, no_feed]

#Count %
summ['PassedQC(%)'] = round(100*summ.PassedQC/summ.Total, 3)
summ['Host(%_of_PassedQC)'] = round(100*(summ.PassedQC-summ.NoHost)/summ.PassedQC, 3)
summ['Feed(%_of_PassedQC)'] = round(100*(summ.NoHost-summ.NoFeed)/summ.PassedQC, 3)


#add owner of samples
names = pd.read_csv(f'{WS}/sampleName_clientId.txt', sep='\t', index_col='clientId')
summ['Owner'] = names['Owner']
summ.to_csv(f'{WS}/{DATA}/QC_stats.tsv', sep='\t')
summ

,Total,PassedQC,NoHost,NoFeed,PassedQC(%),Host(%_of_PassedQC),Feed(%_of_PassedQC),Owner
SampleID,,,,,,,,
UHO1,35831867.0,35492845.0,34944907.0,34942341.0,99.054,1.544,0.007,Money
UHO10,33592806.0,33285677.0,33181082.0,33169124.0,99.086,0.314,0.036,Money
UHO100,35201028.0,35184245.0,35159077.0,35156321.0,99.952,0.072,0.008,Money
UHO101,35715119.0,35434487.0,35373052.0,35258757.0,99.214,0.173,0.323,Money
UHO102,33628784.0,33391514.0,33276130.0,33272830.0,99.294,0.346,0.010,Money
...,...,...,...,...,...,...,...,...
UHO95,34632053.0,34323601.0,34297988.0,34286021.0,99.109,0.075,0.035,Money
UHO96,41129500.0,40722799.0,40715886.0,40704007.0,99.011,0.017,0.029,Money
UHO97,41883780.0,41511029.0,41507829.0,41505113.0,99.110,0.008,0.007,Money


# Metagenome assemblies Megahit

In [315]:
THREADS = 12
RAM = 90
logdir = f'{LOGS}/MegaHit'

todo = '''
conda activate qiime2-shotgun-2024.2
echo "Processing {BATCH}"
cd $WS
mkdir -p $MH

sras=({IDS})
for name in ${{sras[@]}}; do
    echo
    echo "############################################################"
    echo "Processing $name "
    
    echo "Check if reads already assembled" 
    if [ -f $ASS/${{name}}.fa ]; then
        echo "$name already assembled. Skipping."
        continue
    fi
    echo "$name is not assembled. Attempting assembly..."
    
    echo "Check if reads are paired or single" 
    if [ -f $CR/${{name}}_1.fq.gz ]; then
        echo "$name are paired reads."
        CR1=$CR/${{name}}_1.fq.gz
        CR2=$CR/${{name}}_2.fq.gz
    
        megahit -1 $CR1 -2 $CR2 -o $MH/$name -t {THREADS} --presets meta-sensitive
    fi
    
    if [ -f $CR/${{name}}.fq.gz ]; then
        echo "$name are single reads."
        CR1=$CR/${{name}}.fq.gz
    
        megahit -r $CR1 -o $MH/$name -t {THREADS} --presets meta-sensitive
    fi

    echo "Checking if assembly created"
    ass=$MH/$name/final.contigs.fa
    if [ -f $ass ]; then
        echo "Moving assembly"
        mv $ass $ASS/${{name}}.fa
    else
        echo "There is something wrong with assembly"
    fi
    rm -r $MH/$name
done

echo
echo "############################################################"
echo "###                   All done! Enjoy!                   ###"
echo "### Citations:                                           ###"
echo "### 1. MegaHit:                                          ###"
echo "###    https://doi.org/10.1093/bioinformatics/btv033     ###"
echo "############################################################"   
'''

#launch samples
for batch, ids in batches.items():
    if batch in ['batch1-30']:
        continue

    IDS = ' '.join(ids)
    binac(ppn=THREADS, time='07:00:00:00', mem=RAM, name=f'{batch}_MH', queue='long', 
          logdir=logdir, todo=todo.format(BATCH=batch, IDS=IDS, THREADS=THREADS))

11376483
11376484
11376485
11376486
11376487
11376488
11376489
11376490
11376491
11376492
11376493
11376494
11376495
11376496
11376497
11376498
11376499
11376500
11376501
11376502
11376503
11376504
11376505
11376506
11376507
11376508
11376509
11376510
11376511
11376513
11376514
11376515
11376516
11376517
11376518
11376519
11376520
11376521
11376522
11376523
11376524
11376525
11376526
11376527
11376528
11376529
11376530
11376531
11376532
11376533


In [237]:
sras

['SRR12730711',
 'SRR12730712',
 'SRR12730713',
 'SRR12730714',
 'SRR12730715',
 'SRR12730716',
 'SRR12730718',
 'SRR12730719',
 'SRR12730720',
 'SRR12730721',
 'SRR12730722',
 'SRR12730723',
 'SRR12730724',
 'SRR12730725',
 'SRR12730727',
 'SRR12730728',
 'SRR12730729',
 'SRR12730730',
 'SRR12730731',
 'SRR12730732',
 'SRR12730733',
 'SRR12730734',
 'SRR12730735',
 'SRR12730736',
 'SRR12730737',
 'SRR12730738',
 'SRR12730739',
 'SRR12730740',
 'SRR12730741',
 'SRR12730742',
 'SRR12730744',
 'SRR12730745',
 'SRR12730746',
 'SRR12730747',
 'SRR12730748',
 'SRR12730749',
 'SRR12730750',
 'SRR8840688',
 'SRR8840689',
 'SRR8840690',
 'SRR8840691',
 'SRR8840692',
 'SRR8840693',
 'SRR8840694',
 'SRR6323082',
 'SRR6323083',
 'SRR6323084',
 'SRR6323085',
 'SRR6323086',
 'SRR6323087',
 'SRR6323088',
 'SRR6323089',
 'SRR6323090',
 'SRR6323091',
 'SRR6323092',
 'SRR6323093',
 'SRR6323094',
 'SRR6323095',
 'SRR6323096',
 'SRR6323097',
 'SRR6323098',
 'SRR6323099',
 'SRR6323100',
 'SRR6323101',
 'S

In [ ]:
#quast -t {THREADS} -o {out}/Quast {out}/final.contigs.fa

In [5]:
for clean in cleans:
    out = f'{WS}/Results/MegaHit/{clean}'
    !rm -r $out/intermediate_contigs
    

In [4]:
#zip assemblies

logs = f"{LOGS}/MegaHit"

todo = '''
conda activate Work
cd $WS

zip -r MegaHit.zip Results/MegaHit

'''

binac(time='00:48:00:00', mem=16, name=f'MegaHit_zip', queue='short', logdir=logs,
     todo=todo)

11242915


## Metadata

In [36]:
#manifest file
md = pd.DataFrame()
for sample in cleans:
    md.loc[len(md),['SampleID', 'Path']] = [sample, f'{WS}/{CR}/{sample}_1.fq.gz']
    md.loc[len(md),['SampleID', 'Path']] = [sample, f'{WS}/{CR}/{sample}_2.fq.gz']

mds = np.array_split(md, 44)
for i, df in enumerate(mds):
    df.to_csv(f'{WS}/{DATA}/mudoger_meta_{i}.tsv', sep='\t', index=False, header=False)

## Module 1. Preprocess

In [7]:
# Did not work due to .gz files. Should be unzipped
THREADS = 6
RAM = 120
logdir = f'{LOGS}/MuDoGeR'

todo = '''

cd $WS
mkdir -p Results/MuDoGeR_out
$SOURCE/mudoger_singularity.sh preprocess \
    -s $SOURCE/mudogerV1.sif \
    -o Results/MuDoGeR_out \
    -i Data \
    -d $DB/MuDoGeR_db \
    -m {RAM} \
    -t {THREADS} \
    -f mudoger_meta_{i}.tsv
    
'''

for i in range(44):
    binac(ppn=THREADS, time='07:00:00:00', mem=RAM, name=f'MuDoGeR_{i}', queue='long', 
              logdir=logdir, todo=todo.format(RAM=RAM, THREADS=THREADS, i=i))



11257792
11257793
11257794
11257795
11257796
11257797
11257798
11257799
11257800
11257801
11257802
11257803
11257804
11257805
11257806
11257807
11257808
11257809
11257810
11257811
11257812
11257813
11257814
11257815
11257816
11257817
11257818
11257819
11257820
11257821
11257822
11257823
11257824
11257825
11257826
11257827
11257828
11257829
11257830



KeyboardInterrupt



# Status

In [21]:
!qstat -u ho_graaf20


mgmt02: 
                                                                                  Req'd       Req'd       Elap
Job ID                  Username    Queue    Jobname          SessID  NDS   TSK   Memory      Time    S   Time
----------------------- ----------- -------- ---------------- ------ ----- ------ --------- --------- - ---------
11327685                ho_graaf20  smp      BLAST-taxa.sh     28410     1     24    1000gb 720:00:00 R 568:34:03
11339514                ho_graaf20  smp      SM_MAGs.sh        39589     1     12     256gb 720:00:00 R 405:57:55
11376500                ho_graaf20  long     batch541-570_MH.   8585     1     12      90gb 168:00:00 R 139:24:08
11376501                ho_graaf20  long     batch571-600_MH.   8390     1     12      90gb 168:00:00 R 136:26:16
11376502                ho_graaf20  long     batch601-630_MH.  31769     1     12      90gb 168:00:00 R 135:51:01
11376503                ho_graaf20  long     batch631-660_MH.   1157     1     12   

In [108]:
!qdel 11366477

In [313]:
kill =  """
11376394                ho_graaf20  long     batch31-60_MH.sh    --      1     24     120gb 168:00:00 Q       -- 
11376395                ho_graaf20  long     batch61-90_MH.sh    --      1     24     120gb 168:00:00 Q       -- 
11376396                ho_graaf20  long     batch91-120_MH.s    --      1     24     120gb 168:00:00 Q       -- 
11376397                ho_graaf20  long     batch121-150_MH.    --      1     24     120gb 168:00:00 Q       --  
"""

for job in kill.split('\n'):
    ID = job.split(' ')[0]
    !qdel $ID

usage: qdel [{ -a | -c | -p | -t | -W delay | -m message}] [-b retry_seconds] [<JOBID>[<JOBID>]|'all'|'ALL']...
       -a -c, -m, -p, -t, and -W are mutually exclusive
usage: qdel [{ -a | -c | -p | -t | -W delay | -m message}] [-b retry_seconds] [<JOBID>[<JOBID>]|'all'|'ALL']...
       -a -c, -m, -p, -t, and -W are mutually exclusive


In [78]:
!qstat 11366127  


qstat: Unknown Job Id Error 11366127.mgmt


In [4]:
!qstat

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
10161757[]                 fb_rna(1)        ho_braat76             0 C short          
10899958                   h2681L14         hd_wu263               0 H long           
11089184                   STDIN            tu_zxmle82             0 Q gpu            
11151211                   Cluster_reads.sh ho_graaf20      3871:14: R smp            
11153399                   adec_0           st_uws86178     2610:53: R long           
11153400                   adec_1           st_uws86178     343:09:4 R long           
11153405                   adec_0           st_uws86178     2607:08: R long           
11153406                   adec_1           st_uws86178     341:05:3 R long           
11154060                   ...-tuebingen.de tu_emins01      556:45:2 R gpu            
11154619                   go_allLambda.sh  st_st176732     1710: